In [ ]:
from langchain_groq import ChatGroq
from dotenv import load_dotenv
import os
load_dotenv()

In [10]:
groq_api_key = os.environ.get('GROQ_API_KEY')

In [ ]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///Chinook.db")
print(db.dialect)
print(db.get_usable_table_names())
db.run("SELECT * FROM Artist LIMIT 10;")

In [ ]:
from langchain.chains import create_sql_query_chain

llm = ChatGroq(model="llama3-70b-8192", temperature=0.5)
chain = create_sql_query_chain(llm, db)
response = chain.invoke({"question": "How many employees are there."})
response

In [ ]:
db.run(response.split(':')[-1])

In [ ]:
chain.get_prompts()[0].pretty_print()

In [ ]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool

execute_query = QuerySQLDataBaseTool(db=db)
write_query = create_sql_query_chain(llm, db)
chain = write_query
db.run(chain.invoke({"question": "How many employees are there"}).split(':')[-1])

In [ ]:
from operator import itemgetter

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

answer_prompt = PromptTemplate.from_template(
    """Given the following user question, corresponding SQL query, and SQL result, answer the user question.

Question: {question}
SQL Query: {query}
SQL Result: {result}
Answer: """
)

answer = answer_prompt | llm | StrOutputParser()
chain = (
    RunnablePassthrough.assign(query=write_query).assign(
        result=itemgetter("query") | execute_query
    )
    | answer
)

chain.invoke({"question": "How many employees are there"})

In [ ]:
RunnablePassthrough.assign(query=write_query).assign(
        result=itemgetter("query") | execute_query
    )

In [31]:
from langchain_community.agent_toolkits import create_sql_agent

agent_executor = create_sql_agent(llm, db=db, agent_type="openai-tools", verbose=True)

In [ ]:
agent_executor.invoke(
    {
        "input": "List the total sales per country. Which country's customers spent the most?"
    }
)

In [ ]:
agent_executor.invoke({"input": "Describe the playlisttrack table"})